In [1]:
from libraries import *
from parameters import *
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.model_selection import cross_val_score
from numpy import asarray
from numpy import savetxt

import scvi

%matplotlib inline

In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
adata = sc.read('outputs/anndata/adata-hash-features_ALLKO_ForSCVI_12222020.h5ad')


In [33]:
fBarMat_grouped = adata.obs[["G_1","G_2", "G_3", "G_4", "G_5", "G_6", "G_7" ,"G_CONTROL"]]

In [34]:
fBarMat_grouped

,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_CONTROL
AAACGAAAGCAGTCTT-1-E3-1-0,0,0,1,0,0,0,0,0
AAACGAATCTCAACCC-1-E3-1-0,0,1,0,0,0,0,0,0
AAACGCTAGTTGTAGA-1-E3-1-0,0,0,0,0,0,0,0,1
AAACGCTCAAAGACGC-1-E3-1-0,1,0,0,0,0,0,0,0
AAACGCTCAGTGTACT-1-E3-1-0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
TTGTGGATCTATTTCG-1-E4-24-1,0,0,0,1,1,0,0,0
TTGTGTTCAGGCGATA-1-E4-24-1,0,1,0,0,0,1,0,0
TTGTTCAGTCCGGATC-1-E4-24-1,0,0,0,0,0,1,1,0
TTTCAGTTCTATCGGA-1-E4-24-1,0,0,1,0,0,0,1,0


In [12]:
fBarMat_grouped.columns[0:7]

Index(['G_1', 'G_2', 'G_3', 'G_4', 'G_5', 'G_6', 'G_7'], dtype='object')

In [13]:
fBarMat_groupedSingle = fBarMat_grouped[fBarMat_grouped.columns[0:7]].apply(lambda x: ''.join(x.dropna().astype(str)),axis=1)


In [26]:
fBarMat_groupedSingle.values

array(['0010000', '0100000', '0000000', ..., '0000011', '0010001',
       '1000000'], dtype=object)

In [35]:
adata.obs['fBarMat_groupedSingle'] = fBarMat_groupedSingle.values

In [36]:
adata.obs['fBarMat_groupedSingle']

AAACGAAAGCAGTCTT-1-E3-1-0     0010000
AAACGAATCTCAACCC-1-E3-1-0     0100000
AAACGCTAGTTGTAGA-1-E3-1-0     0000000
AAACGCTCAAAGACGC-1-E3-1-0     1000000
AAACGCTCAGTGTACT-1-E3-1-0     0000100
                               ...   
TTGTGGATCTATTTCG-1-E4-24-1    0001100
TTGTGTTCAGGCGATA-1-E4-24-1    0100010
TTGTTCAGTCCGGATC-1-E4-24-1    0000011
TTTCAGTTCTATCGGA-1-E4-24-1    0010001
TTTGACTCAGATCCAT-1-E4-24-1    1000000
Name: fBarMat_groupedSingle, Length: 131361, dtype: object

In [37]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
fBarMat_encodedDF = pd.Series(label_encoder.fit_transform(fBarMat_groupedSingle))


In [39]:
tp = pd.DataFrame(data={"fBarMat_groupedSingle" :fBarMat_groupedSingle.values,
             "fBarMat_encodedDF" : fBarMat_encodedDF.values})
tp = tp.drop_duplicates()
tp

,fBarMat_groupedSingle,fBarMat_encodedDF
0,0010000,14
1,0100000,24
2,0000000,0
3,1000000,40
4,0000100,4
...,...,...
124027,0011010,22
124092,1001100,50
124717,1000011,43
125533,0101010,33


In [ ]:
for i in range(1,8):
    adata.obs[f'G_{i}'] = adata.obs[f'G_{i}'].astype(str)

In [ ]:
adata.obs[f'G_CONTROL'] = adata.obs[f'G_CONTROL'].astype(str)

In [ ]:
zs = ["G_1","G_2", "G_3", "G_4", "G_5", "G_6", "G_7" ,"G_CONTROL"]
sc.pl.umap(adata, color=zs, ncols=4, palette=["grey", "red"], groups="1")

In [40]:
sc.tl.embedding_density(adata, groupby="fBarMat_groupedSingle")

... storing 'fBarMat_groupedSingle' as categorical


computing density on 'umap'


ValueError: More than 10 categories in 'fBarMat_groupedSingle' column.

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_1", group="1", color_map="nipy_spectral",title="G_1")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_2", group="1", color_map="nipy_spectral", title="G_2")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_3", group="1", color_map="nipy_spectral", title="G_3")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_4", group="1", color_map="nipy_spectral", title="G_4")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_5", group="1", color_map="nipy_spectral", title="G_5")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_6", group="1", color_map="nipy_spectral", title="G_6")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_7", group="1", color_map="nipy_spectral", title="G_7")

In [ ]:
sc.pl.embedding_density(adata, key="umap_density_G_CONTROL", group="1", color_map="nipy_spectral", title="G_CONTROL")

In [ ]:
adataSubType0 = adata[adata.obs['SubCellType_0'] == 1,]

In [ ]:
adataSubType0.X.shape

In [ ]:
sc.tl.embedding_density(adataSubType0, groupby="G_1")
sc.tl.embedding_density(adataSubType0, groupby="G_2")
sc.tl.embedding_density(adataSubType0, groupby="G_3")
sc.tl.embedding_density(adataSubType0, groupby="G_4")
sc.tl.embedding_density(adataSubType0, groupby="G_5")
sc.tl.embedding_density(adataSubType0, groupby="G_6")
sc.tl.embedding_density(adataSubType0, groupby="G_7")
sc.tl.embedding_density(adataSubType0, groupby="G_CONTROL")


In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_1", group="1", color_map="nipy_spectral",title="G_1")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_2", group="1", color_map="nipy_spectral",title="G_2")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_3", group="1", color_map="nipy_spectral",title="G_3")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_4", group="1", color_map="nipy_spectral",title="G_4")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_5", group="1", color_map="nipy_spectral",title="G_5")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_6", group="1", color_map="nipy_spectral",title="G_6")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_7", group="1", color_map="nipy_spectral",title="G_7")

In [ ]:
sc.pl.embedding_density(adataSubType0, key="umap_density_G_CONTROL", group="1", color_map="nipy_spectral",title="G_CONTROL")

In [ ]:
adataSubType1 = adata[adata.obs['SubCellType_1'] == 1,]

In [ ]:
sc.tl.embedding_density(adataSubType1, groupby="G_1")
sc.tl.embedding_density(adataSubType1, groupby="G_2")
sc.tl.embedding_density(adataSubType1, groupby="G_3")
sc.tl.embedding_density(adataSubType1, groupby="G_4")
sc.tl.embedding_density(adataSubType1, groupby="G_5")
sc.tl.embedding_density(adataSubType1, groupby="G_6")
sc.tl.embedding_density(adataSubType1, groupby="G_7")
sc.tl.embedding_density(adataSubType1, groupby="G_CONTROL")


In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_1", group="1", color_map="nipy_spectral",title="G_1")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_2", group="1", color_map="nipy_spectral",title="G_2")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_3", group="1", color_map="nipy_spectral",title="G_3")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_4", group="1", color_map="nipy_spectral",title="G_4")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_5", group="1", color_map="nipy_spectral",title="G_5")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_6", group="1", color_map="nipy_spectral",title="G_6")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_7", group="1", color_map="nipy_spectral",title="G_7")

In [ ]:
sc.pl.embedding_density(adataSubType1, key="umap_density_G_CONTROL", group="1", color_map="nipy_spectral",title="G_CONTROL")